In [163]:
import matplotlib.pyplot as plt
from keras.layers import Dense, Flatten, Dropout
from keras.layers.recurrent import LSTM
from keras.models import Sequential
import time
import numpy as np
import quandl
quandl.ApiConfig.api_key = 'vcx4s9_juRgoyVT9EL-M' 
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import pandas as pd

In [164]:
def normalise_data(dataset):
    scaler = MinMaxScaler(feature_range=(0, 1))
    dataset = scaler.fit_transform(dataset)
    return dataset

def load_data(quandl_id):
    data  = quandl.get(quandl_id)
    print data.columns
    data = data.reset_index(drop=False)
    return data.values

def prepare_sequences(x_train, y_train, window_length):
    windows = []
    windows_y = []
    for i, sequence in enumerate(x_train):
        len_seq = len(sequence)
        for window_start in range(0, len_seq - window_length + 1):
            window_end = window_start + window_length
            window = sequence[window_start:window_end]
            windows.append(window)
            windows_y.append(y_train[i])
    return np.array(windows), np.array(windows_y)

def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back)]
        dataX.append(a)
        dataY.append(dataset[i + look_back])
    return np.array(dataX), np.array(dataY)

In [165]:
seq_length = 50
features_length = 7
input_shape = (seq_length, features_length)

#Step 1 Load and Normalize data
dataset = load_data("BCHARTS/BITSTAMPUSD")
# timestamps = dataset[:,0]
normalised_values = normalise_data(dataset[:,1:8])

seq_length = 50
sequences = [normalised_values[x:x + seq_length] for x in xrange(0, len(normalised_values), seq_length)]

Index([u'Open', u'High', u'Low', u'Close', u'Volume (BTC)',
       u'Volume (Currency)', u'Weighted Price'],
      dtype='object')


In [166]:
# split into train and test sets
train_size = int(len(sequences) * 0.90)
test_size = len(sequences) - train_size
train, test = sequences[0:train_size], sequences[train_size:len(sequences)]

# reshape into X=t and Y=t+1
look_back = 1
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

In [167]:
# reshape input to be [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], 50, trainX.shape[3]))
testX = np.reshape(testX, (testX.shape[0], 50, testX.shape[3]))

In [168]:
X, Y = prepare_sequences(trainX, trainY, 50)

In [169]:
#Step 2 Build Model
model = Sequential()
model.add(LSTM(128, return_sequences=True, input_shape=input_shape, dropout_W=0.5, dropout_U=0.5))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes, activation='softmax'))

start = time.time()
model.compile(loss='mse', optimizer='rmsprop')
print 'compilation time : ', time.time() - start

NameError: name 'nb_classes' is not defined

In [161]:
#Step 3 Train the model
model.fit(X, Y, batch_size=16, nb_epoch=1, validation_split=0.05)

ValueError: Error when checking model target: expected dropout_16 to have 2 dimensions, but got array with shape (35, 50, 7)

In [140]:
# Print Close values
# plt.plot(normalised_values[:,3])
# plt.show()

#Step 4 - Plot the predictions!
# predictions = lstm.predict_sequences_multiple(model, X_test, 50, 50)
# lstm.plot_results_multiple(predictions, y_test, 50)